# Histogram matching
Utility scripts for batch histogram matching of photos. Histogram matching can be used to normalize differences in lighting between photos.

In [1]:
import os
import numpy as np
from skimage import io
from skimage.exposure import match_histograms

from utils import sprint, list_paths, add_suffix

## Matching function

In [2]:
def histogram_match_photos(photo_paths, write=True, save_dir=None, save_suffix="_match", reference_index=0, dtype=None, silent=False):
    """
    Histogram match a set of photos, using one of the photos as a reference. Return the matched photos
    and optionally write to files.
    @param {list} photo_paths A list of relative or absolute paths to image files
    @param {bool, default True} write If true, the matched photos will be written to disk.
    @param {str, default None} save_dir A directory to write matched files to. If none is provided,
    files will be written to the directory they were read from.
    @param {str, default "_match"} saveSuffix Characters to append to the filename when writing
    @param {int, default 0} reference_index The index of the photo path to use as a reference when
    histogram matching.
    @param {numpy.dtype, default None} dtype The data type to cast values of the matched photo to.
    If none is provided, the data type of the input reference image will be used.
    @return {list} List of np.ndarrays representing histogram matched photos
    """
    photos = [io.imread(path) for path in photo_paths]
    reference = photos[reference_index]
    
    if not dtype:
        dtype = reference.dtype
    
    matched_photos = []
    
    for i, photo in enumerate(photos):
        sprint(f"Matching photo {i + 1} of {len(photos)}...", silent)
        
        matched = match_histograms(photo, reference).astype(dtype)
        matched_photos.append(matched)
        
        if write:
            in_path = photo_paths[i]
            out_path = os.path.join(save_dir, os.path.basename(in_path)) if save_dir else in_path
            out_path = add_suffix(out_path, save_suffix)
            sprint(f"Writing matched photo to {out_path}...", silent)
            io.imsave(out_path, matched)
    
    return matched_photos

## Example

In [5]:
img_dir = os.path.join("data")
valid_extensions = (".tif")
photo_paths = list_paths(img_dir, valid_extensions)

matched_photos = histogram_match_photos(photo_paths, write=True, save_suffix="_matched", silent=True)